<b> <font size = 5>Data Cleaning - Facility Level Data </b> </font>

This iPython notebook loads the raw data from the national emissions inventory and performs data cleaning steps to get the facility level emissions data for PM25 and NO2. The raw dataset contains emissions for all facilities in Alameda county, which is further fitlered down to get facilities in Oakland, Berkeley, Alameda, Piedmon and San Leandro. 

In [2]:
#Import basic python packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import descartes
import geopandas as gpd
from shapely.geometry import Point, Polygon
from shapely.ops import nearest_points

import seaborn as sns

from mpl_toolkits.axes_grid1 import make_axes_locatable

import math

import time

from matplotlib import cm

import matplotlib.lines as mlines

%matplotlib inline

# <b> <font size =5>Load Raw Facility Data From National Emissions Inventory for Alameda County </b> </font>

In [2]:
# Read data for Alameda County and drop duplicate facility ids
oak_facility = pd.read_csv('Data/oakland_facility_2017.csv', header = 0)
# Display all columns in the dataframe
oak_facility.columns

Index(['Unnamed: 0', 'county', 'eis facility id', 'agency facility id',
       'tri facility id', 'company name', 'site name', 'facility source type',
       'site latitude', 'site longitude', 'address', 'city', 'zip code',
       'pollutant code', 'pollutant type(s)', 'total emissions',
       'emissions uom', 'data set'],
      dtype='object')

In [3]:
# Drop columns that are not needed 
oak_facility.drop(columns = ['agency facility id', 'tri facility id','pollutant type(s)', 'data set',
                             'site name', 'address','Unnamed: 0', 'county'], inplace=True)

In [4]:
# Rename site latitude and site longitude column to just latitude and logitude
oak_facility.rename(columns={"site latitude": "latitude", "site longitude": "longitude", 'facility source type':'source-category'}, inplace=True)

In [5]:
# Display all columns in the dataframe
oak_facility.columns

Index(['eis facility id', 'company name', 'source-category', 'latitude',
       'longitude', 'city', 'zip code', 'pollutant code', 'total emissions',
       'emissions uom'],
      dtype='object')

In [6]:
# Filter the dataframe further for sources/facilities only in Oakland and neighbouring cities like San Leandro, Piedmont, Alameda and Berkeley
oak_facility = oak_facility[(oak_facility['city'] == 'OAKLAND') | (oak_facility['city'] == 'SAN LEANDRO') | (oak_facility['city'] == 'BERKELEY')|
                           (oak_facility['city'] == 'ALAMEDA') | (oak_facility['city'] == 'PIEDMONT')]

In [7]:
oak_facility.head(50)

,eis facility id,company name,source-category,latitude,longitude,city,zip code,pollutant code,total emissions,emissions uom
3,126911,PACIFIC BELL,NaN,37.76685,-122.24972,ALAMEDA,94501,NOX,0.054659,TON
4,126911,PACIFIC BELL,NaN,37.76685,-122.24972,ALAMEDA,94501,PM10-PRI,0.000749,TON
5,126911,PACIFIC BELL,NaN,37.76685,-122.24972,ALAMEDA,94501,PM25-PRI,0.000731,TON
9,127111,PACIFIC BELL,NaN,37.86771,-122.26784,BERKELEY,94704,NOX,0.162383,TON
10,127111,PACIFIC BELL,NaN,37.86771,-122.26784,BERKELEY,94704,PM10-PRI,0.002226,TON
11,127111,PACIFIC BELL,NaN,37.86771,-122.26784,BERKELEY,94704,PM25-PRI,0.002173,TON
18,127511,PACIFIC BELL,NaN,37.80603,-122.26946,OAKLAND,94612,NOX,0.619550,TON
19,127511,PACIFIC BELL,NaN,37.80603,-122.26946,OAKLAND,94612,PM10-PRI,0.008494,TON
20,127511,PACIFIC BELL,NaN,37.80603,-122.26946,OAKLAND,94612,PM25-PRI,0.008291,TON
21,127611,PACIFIC BELL,NaN,37.78422,-122.22241,OAKLAND,94601,NOX,0.052783,TON


In [8]:
### Clean up source-category column
oak_facility['source-category'] = oak_facility['source-category'].replace({'Institutional (school, hospital, prison, etc.)':'Institution', 'Breweries/Distilleries/Wineries':'Breweries-Distilleries-Wineries'})
oak_facility.replace(' ', '-',regex=True,inplace=True)
oak_facility.replace(',', '-',regex=True,inplace=True)
oak_facility.replace('---', '-',regex=True,inplace=True)
oak_facility.replace('--', '-',regex=True,inplace=True)
oak_facility.replace(np.nan, 'Unknown', regex= True, inplace=True)

In [9]:
oak_facility['source-category'].unique()


array(['Unknown', 'Institution', 'Electricity-Generation-via-Combustion',
       'Concrete-Batch-Plant', 'Hot-Mix-Asphalt-Plant',
       'Wastewater-Treatment-Facility', 'Crematory-Human',
       'Food-Products-Processing-Plant',
       'Breweries-Distilleries-Wineries',
       'Auto-Body-Painting-or-Repair-Shop', 'Foundries-Iron-and-Steel',
       'Fiberglass-Products-Manuf', 'Bakeries',
       'Dry-Cleaner-Petroleum-Solvent', 'Pharmaceutical-Manufacturing',
       'Landfill', 'Airport', 'Concrete-Block-Pipe-or-Other-Manuf'],
      dtype=object)

# <b> <font size = 5>  Create NO2 Dataset </b> </font>

In [10]:
## Creating a separate dataframe for NOx
Facility_NO2 = oak_facility[oak_facility['pollutant code'] == 'NOX']

In [11]:
Facility_NO2.shape

(328, 10)

In [12]:
Facility_NO2['city'].unique()

array(['ALAMEDA', 'BERKELEY', 'OAKLAND', 'SAN-LEANDRO', 'PIEDMONT'],
      dtype=object)

In [13]:
Facility_NO2.drop(columns = ['pollutant code', 'zip code','emissions uom'], inplace=True)

/home/shridhar/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [14]:
Facility_NO2.head()

,eis facility id,company name,source-category,latitude,longitude,city,total emissions
3,126911,PACIFIC-BELL,Unknown,37.76685,-122.24972,ALAMEDA,0.054659
9,127111,PACIFIC-BELL,Unknown,37.86771,-122.26784,BERKELEY,0.162383
18,127511,PACIFIC-BELL,Unknown,37.80603,-122.26946,OAKLAND,0.619550
21,127611,PACIFIC-BELL,Unknown,37.78422,-122.22241,OAKLAND,0.052783
24,127711,PACIFIC-BELL,Unknown,37.83336,-122.26307,OAKLAND,0.116122


##### Write into csv file without source category information
Facility_NO2.to_csv("Data/Unused-data/Facilities_NO2_all_wo_srccat.csv")

**Some missing source categories were filled in manually by using the Latitude and Longitude information to locate the facilities on Google Maps or Google Earth**

## <b> <font size = 4> Load NO2 dataset with source categories filled in manually </b> </font>

In [16]:
Facility_NO2 = pd.read_csv("Data/Unused-data/Facilities_NO2_all.csv")

In [17]:
Facility_NO2.drop(columns = ['Unnamed: 0'], inplace=True)


In [18]:
Facility_NO2.tail()

,eis facility id,company name,source-category,latitude,longitude,city,total emissions
323,18697911,Unknown,Wholesale,37.744311,-122.190515,OAKLAND,0.803283
324,18698311,Unknown,Metals,37.717160,-122.181590,SAN-LEANDRO,0.124030
325,18776511,Unknown,Manufacturing,37.853018,-122.292710,BERKELEY,0.455809
326,18779811,Unknown,Manufacturing,37.707820,-122.185500,SAN-LEANDRO,0.000309
327,18781211,Unknown,Unknown,37.812106,-122.186447,OAKLAND,0.001603


In [19]:
Facility_NO2['source-category'].unique()

array(['Institution', 'Electricity-Generation-via-Combustion', 'Unknown',
       'Manufacturing', 'Waste-Recycling', 'Metals',
       'Auto-Body-Painting-or-Repair-Shop', 'Hot-Mix-Asphalt-Plant',
       'Wastewater-Treatment-Facility', 'Crematory-Human',
       'Food-Products-Processing-Plant',
       'Breweries-Distilleries-Wineries', 'Foundries-Iron-and-Steel',
       'Port', 'Bakeries', 'Dry-Cleaner-Petroleum-Solvent', 'Wholesale',
       'Retail', 'Transportation', 'Residential', 'Airport',
       'Pharmaceutical-Manufacturing', 'Landfill', 'USCG', 'Parking',
       'DataCenter', 'GasStation', 'Stadium'], dtype=object)

In [20]:
#Create a new column as source type and fill it in with source category
Facility_NO2['source-type'] = Facility_NO2['source-category']

**Update source type names with shorter names**

In [21]:
### Update source type names with shorter names that are easier to follow

Facility_NO2['source-type'].replace('Electricity-Generation-via-Combustion', 'EGen',regex=True,inplace=True)
Facility_NO2['source-type'].replace('Auto-Body-Painting-or-Repair-Shop', 'AutoRepair',regex=True,inplace=True)
Facility_NO2['source-type'].replace('Wastewater-Treatment-Facility', 'WWTP',regex=True,inplace=True)
Facility_NO2['source-type'].replace('Concrete-Batch-Plant', 'ConcretePlant',regex=True,inplace=True)
Facility_NO2['source-type'].replace('Hot-Mix-Asphalt-Plant', 'AsphaltPlant',regex=True,inplace=True)
Facility_NO2['source-type'].replace('Crematory-Human', 'Crematory',regex=True,inplace=True)
Facility_NO2['source-type'].replace('Breweries-Distilleries-Wineries', 'Breweries',regex=True,inplace=True)
Facility_NO2['source-type'].replace('Foundries-Iron-and-Steel', 'Foundaries',regex=True,inplace=True)
Facility_NO2['source-type'].replace('Fiberglass-Products-Manuf', 'FiberGlass',regex=True,inplace=True)
Facility_NO2['source-type'].replace('Dry-Cleaner-Petroleum-Solvent', 'Solvent',regex=True,inplace=True)
Facility_NO2['source-type'].replace('Pharmaceutical-Manufacturing', 'Pharma',regex=True,inplace=True)
Facility_NO2['source-type'].replace('Food-Products-Processing-Plant', 'FoodPlant',regex=True,inplace=True)
Facility_NO2['source-type'].replace('Concrete-Block-Pipe-or-Other-Manuf', 'PipePlant',regex=True,inplace=True)


In [22]:
Facility_NO2['source-type'].unique()

array(['Institution', 'EGen', 'Unknown', 'Manufacturing',
       'Waste-Recycling', 'Metals', 'AutoRepair', 'AsphaltPlant', 'WWTP',
       'Crematory', 'FoodPlant', 'Breweries', 'Foundaries', 'Port',
       'Bakeries', 'Solvent', 'Wholesale', 'Retail', 'Transportation',
       'Residential', 'Airport', 'Pharma', 'Landfill', 'USCG', 'Parking',
       'DataCenter', 'GasStation', 'Stadium'], dtype=object)

## <b> <font size = 4>  Dropping sources with 'Unknown' source category </b> </font>

In [23]:
Facility_NO2 = Facility_NO2[Facility_NO2['source-category'] != 'Unknown']

In [24]:
Facility_NO2['source-type'].unique()

array(['Institution', 'EGen', 'Manufacturing', 'Waste-Recycling',
       'Metals', 'AutoRepair', 'AsphaltPlant', 'WWTP', 'Crematory',
       'FoodPlant', 'Breweries', 'Foundaries', 'Port', 'Bakeries',
       'Solvent', 'Wholesale', 'Retail', 'Transportation', 'Residential',
       'Airport', 'Pharma', 'Landfill', 'USCG', 'Parking', 'DataCenter',
       'GasStation', 'Stadium'], dtype=object)

In [25]:
Facility_NO2.shape

(317, 8)

#### Write into a new csv file
Facility_NO2.to_csv("Data/All_NO2_Facilities_Final.csv")

# <b> <font size = 5>  Create PM Dataset </b> </font>

In [27]:
## Creating a separate dataframe for PM/Black carbon
Oak_PM10_25 = oak_facility[oak_facility['pollutant code'] != 'NOX']

In [28]:
PM10_pri = Oak_PM10_25[Oak_PM10_25['pollutant code']=='PM10-PRI']

In [29]:
## Drop PM10-PRI rows in Oak_PM dataframe
Oak_PM25  = Oak_PM10_25[Oak_PM10_25['pollutant code'] == 'PM25-PRI']

In [30]:
## Create Facility_PM dataframe
Facility_PM = pd.merge(Oak_PM25, PM10_pri, left_on ='eis facility id', right_on ='eis facility id', how='inner')


In [31]:
Facility_PM.head()

,eis facility id,company name_x,source-category_x,latitude_x,longitude_x,city_x,zip code_x,pollutant code_x,total emissions_x,emissions uom_x,company name_y,source-category_y,latitude_y,longitude_y,city_y,zip code_y,pollutant code_y,total emissions_y,emissions uom_y
0,126911,PACIFIC-BELL,Unknown,37.76685,-122.24972,ALAMEDA,94501,PM25-PRI,0.000731,TON,PACIFIC-BELL,Unknown,37.76685,-122.24972,ALAMEDA,94501,PM10-PRI,0.000749,TON
1,127111,PACIFIC-BELL,Unknown,37.86771,-122.26784,BERKELEY,94704,PM25-PRI,0.002173,TON,PACIFIC-BELL,Unknown,37.86771,-122.26784,BERKELEY,94704,PM10-PRI,0.002226,TON
2,127511,PACIFIC-BELL,Unknown,37.80603,-122.26946,OAKLAND,94612,PM25-PRI,0.008291,TON,PACIFIC-BELL,Unknown,37.80603,-122.26946,OAKLAND,94612,PM10-PRI,0.008494,TON
3,127611,PACIFIC-BELL,Unknown,37.78422,-122.22241,OAKLAND,94601,PM25-PRI,0.000706,TON,PACIFIC-BELL,Unknown,37.78422,-122.22241,OAKLAND,94601,PM10-PRI,0.000724,TON
4,127711,PACIFIC-BELL,Unknown,37.83336,-122.26307,OAKLAND,94609,PM25-PRI,0.001554,TON,PACIFIC-BELL,Unknown,37.83336,-122.26307,OAKLAND,94609,PM10-PRI,0.001592,TON


In [32]:
# Drop columns that are not needed/repetitive
Facility_PM.drop(columns = ['zip code_x','pollutant code_x', 'emissions uom_x','company name_y','latitude_y','longitude_y',
                      'city_y','zip code_y','pollutant code_y','emissions uom_x', 'source-category_y'], inplace=True)

In [33]:
# Rename columns to meaningful names
Facility_PM.rename(columns={"latitude_x": "latitude", "longitude_x": "longitude", "city_x": "city", 
                       "total emissions_x" : "PM25", "total emissions_y": "PM10", "emissions uom_y": "units", 
                           "source-category_x":"source-category", "company name_x":"company"}, inplace=True)

In [34]:
Facility_PM.head()

,eis facility id,company,source-category,latitude,longitude,city,PM25,PM10,units
0,126911,PACIFIC-BELL,Unknown,37.76685,-122.24972,ALAMEDA,0.000731,0.000749,TON
1,127111,PACIFIC-BELL,Unknown,37.86771,-122.26784,BERKELEY,0.002173,0.002226,TON
2,127511,PACIFIC-BELL,Unknown,37.80603,-122.26946,OAKLAND,0.008291,0.008494,TON
3,127611,PACIFIC-BELL,Unknown,37.78422,-122.22241,OAKLAND,0.000706,0.000724,TON
4,127711,PACIFIC-BELL,Unknown,37.83336,-122.26307,OAKLAND,0.001554,0.001592,TON


In [35]:
Facility_PM.shape

(362, 9)

##### Write into csv file without source category information
Facility_PM.to_csv("Data/Unused-data/Facilities_PM_all_wo_srccat.csv")

**Some missing source categories were filled in manually by using the Latitude and Longitude information to locate the facilities on Google Maps or Google Earth**

## <b> <font size = 4> Load PM dataset with source categories filled in manually </b> </font>

In [37]:
Facility_PM = pd.read_csv("Data/Unused-data/All_PM_Facilities.csv")

In [38]:
Facility_PM.drop(columns = ['Unnamed: 0'], inplace=True)

In [39]:
Facility_PM.tail()

,eis facility id,company,source-category,latitude,longitude,city,PM25,PM10,units
357,18778911,Unknown,Wholesale,37.712960,-122.188680,SAN-LEANDRO,22.124544,22.971518,TON
358,18779811,Unknown,Manufacturing,37.707820,-122.185500,SAN-LEANDRO,0.000015,0.000015,TON
359,18781211,Unknown,Unknown,37.812106,-122.186447,OAKLAND,0.000061,0.000061,TON
360,18782611,Unknown,Waste-Recycling,37.759959,-122.207782,OAKLAND,0.992654,1.030214,TON
361,18787011,Unknown,Manufacturing,37.766939,-122.216336,OAKLAND,0.079014,0.131690,TON


In [40]:
Facility_PM['source-category'].unique()

array(['Institution', 'Electricity-Generation-via-Combustion', 'Unknown',
       'Paint Shop', 'Manufacturing', 'Waste-Recycling', 'Metals',
       'Concrete-Batch-Plant', 'Hot-Mix-Asphalt-Plant',
       'Wastewater-Treatment-Facility', 'Crematory-Human',
       'Food-Products-Processing-Plant',
       'Breweries-Distilleries-Wineries',
       'Auto-Body-Painting-or-Repair-Shop', 'Foundries-Iron-and-Steel',
       'Port', 'Fiberglass-Products-Manuf', 'Bakeries',
       'Dry-Cleaner-Petroleum-Solvent', 'Wholesale', 'Retail',
       'Transportation', 'Residential', 'Airport',
       'Pharmaceutical-Manufacturing', 'Landfill', 'USCG',
       'Concrete-Block-Pipe-or-Other-Manuf', 'Parking', 'GasStation',
       'DataCenter', 'Stadium'], dtype=object)

In [41]:
#Create a new column as source type and fill it in with source category
Facility_PM['source-type'] = Facility_PM['source-category']

**Update source type names with shorter names**

In [42]:
### Update source type names with shorter names that are easier to follow
Facility_PM.replace('Paint Shop', 'Auto-Body-Painting-or-Repair-Shop',regex=True,inplace=True)
Facility_PM['source-type'].replace('Electricity-Generation-via-Combustion', 'EGen',regex=True,inplace=True)
Facility_PM['source-type'].replace('Auto-Body-Painting-or-Repair-Shop', 'AutoRepair',regex=True,inplace=True)
Facility_PM['source-type'].replace('Wastewater-Treatment-Facility', 'WWTP',regex=True,inplace=True)
Facility_PM['source-type'].replace('Concrete-Batch-Plant', 'ConcretePlant',regex=True,inplace=True)
Facility_PM['source-type'].replace('Hot-Mix-Asphalt-Plant', 'AsphaltPlant',regex=True,inplace=True)
Facility_PM['source-type'].replace('Crematory-Human', 'Crematory',regex=True,inplace=True)
Facility_PM['source-type'].replace('Breweries-Distilleries-Wineries', 'Breweries',regex=True,inplace=True)
Facility_PM['source-type'].replace('Foundries-Iron-and-Steel', 'Foundaries',regex=True,inplace=True)
Facility_PM['source-type'].replace('Fiberglass-Products-Manuf', 'FiberGlass',regex=True,inplace=True)
Facility_PM['source-type'].replace('Dry-Cleaner-Petroleum-Solvent', 'Solvent',regex=True,inplace=True)
Facility_PM['source-type'].replace('Pharmaceutical-Manufacturing', 'Pharma',regex=True,inplace=True)
Facility_PM['source-type'].replace('Food-Products-Processing-Plant', 'FoodPlant',regex=True,inplace=True)
Facility_PM['source-type'].replace('Concrete-Block-Pipe-or-Other-Manuf', 'PipePlant',regex=True,inplace=True)


In [43]:
Facility_PM['source-type'].unique()

array(['Institution', 'EGen', 'Unknown', 'AutoRepair', 'Manufacturing',
       'Waste-Recycling', 'Metals', 'ConcretePlant', 'AsphaltPlant',
       'WWTP', 'Crematory', 'FoodPlant', 'Breweries', 'Foundaries',
       'Port', 'FiberGlass', 'Bakeries', 'Solvent', 'Wholesale', 'Retail',
       'Transportation', 'Residential', 'Airport', 'Pharma', 'Landfill',
       'USCG', 'PipePlant', 'Parking', 'GasStation', 'DataCenter',
       'Stadium'], dtype=object)

## <b> <font size = 3>  Dropping sources with 'Unknown' source category</b> </font>

In [44]:
Facility_PM = Facility_PM[Facility_PM['source-category'] != 'Unknown']

In [45]:
Facility_PM['source-type'].unique()

array(['Institution', 'EGen', 'AutoRepair', 'Manufacturing',
       'Waste-Recycling', 'Metals', 'ConcretePlant', 'AsphaltPlant',
       'WWTP', 'Crematory', 'FoodPlant', 'Breweries', 'Foundaries',
       'Port', 'FiberGlass', 'Bakeries', 'Solvent', 'Wholesale', 'Retail',
       'Transportation', 'Residential', 'Airport', 'Pharma', 'Landfill',
       'USCG', 'PipePlant', 'Parking', 'GasStation', 'DataCenter',
       'Stadium'], dtype=object)

In [46]:
Facility_PM.shape

(348, 10)

##### Write to CSV
Facility_PM.to_csv("Data/All_PM_Facilities_Final.csv")

In [48]:
Facility_PM

,eis facility id,company,source-category,latitude,longitude,city,PM25,PM10,units,source-type
0,126911,PACIFIC-BELL,Institution,37.766850,-122.249720,ALAMEDA,0.000731,0.000749,TON,Institution
1,127111,PACIFIC-BELL,Institution,37.867710,-122.267840,BERKELEY,0.002173,0.002226,TON,Institution
2,127511,PACIFIC-BELL,Institution,37.806030,-122.269460,OAKLAND,0.008291,0.008494,TON,Institution
3,127611,PACIFIC-BELL,Institution,37.784220,-122.222410,OAKLAND,0.000706,0.000724,TON,Institution
4,127711,PACIFIC-BELL,Institution,37.833360,-122.263070,OAKLAND,0.001554,0.001592,TON,Institution
...,...,...,...,...,...,...,...,...,...,...
356,18776511,Unknown,Manufacturing,37.853018,-122.292710,BERKELEY,0.010500,0.010500,TON,Manufacturing
357,18778911,Unknown,Wholesale,37.712960,-122.188680,SAN-LEANDRO,22.124544,22.971518,TON,Wholesale
358,18779811,Unknown,Manufacturing,37.707820,-122.185500,SAN-LEANDRO,0.000015,0.000015,TON,Manufacturing
360,18782611,Unknown,Waste-Recycling,37.759959,-122.207782,OAKLAND,0.992654,1.030214,TON,Waste-Recycling
